In [28]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import pairwise_distances 
from scipy.sparse.linalg import svds
from surprise import Reader
from surprise import Dataset
from surprise.model_selection import cross_validate
from surprise import NormalPredictor
from surprise import KNNBasic
from surprise import KNNWithMeans
from surprise import KNNWithZScore
from surprise import KNNBaseline
from surprise import SVD
from surprise import BaselineOnly
from surprise import SVDpp
from surprise import NMF
from surprise import SlopeOne
from surprise import CoClustering
from surprise.accuracy import rmse
from surprise import accuracy
from surprise.model_selection import train_test_split
from surprise import evaluate

# Data Collection and Preperation

In [10]:
df1 = pd.read_csv('beer_reviews.csv')

In [11]:
df1.rename(columns={'beer_name': 'name'}, inplace=True)
df1

,brewery_id,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,beer_style,review_palate,review_taste,name,beer_abv,beer_beerid
0,10325,Vecchio Birraio,1234817823,1.5,2.0,2.5,stcules,Hefeweizen,1.5,1.5,Sausa Weizen,5.0,47986
1,10325,Vecchio Birraio,1235915097,3.0,2.5,3.0,stcules,English Strong Ale,3.0,3.0,Red Moon,6.2,48213
2,10325,Vecchio Birraio,1235916604,3.0,2.5,3.0,stcules,Foreign / Export Stout,3.0,3.0,Black Horse Black Beer,6.5,48215
3,10325,Vecchio Birraio,1234725145,3.0,3.0,3.5,stcules,German Pilsener,2.5,3.0,Sausa Pils,5.0,47969
4,1075,Caldera Brewing Company,1293735206,4.0,4.5,4.0,johnmichaelsen,American Double / Imperial IPA,4.0,4.5,Cauldron DIPA,7.7,64883
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1586609,14359,The Defiant Brewing Company,1162684892,5.0,4.0,3.5,maddogruss,Pumpkin Ale,4.0,4.0,The Horseman's Ale,5.2,33061
1586610,14359,The Defiant Brewing Company,1161048566,4.0,5.0,2.5,yelterdow,Pumpkin Ale,2.0,4.0,The Horseman's Ale,5.2,33061
1586611,14359,The Defiant Brewing Company,1160702513,4.5,3.5,3.0,TongoRad,Pumpkin Ale,3.5,4.0,The Horseman's Ale,5.2,33061
1586612,14359,The Defiant Brewing Company,1160023044,4.0,4.5,4.5,dherling,Pumpkin Ale,4.5,4.5,The Horseman's Ale,5.2,33061


In [12]:
df2 = pd.read_csv('beers.csv', index_col='Unnamed: 0')

In [13]:
df2

,abv,ibu,id,name,style,brewery_id,ounces
0,0.050,NaN,1436,Pub Beer,American Pale Lager,408,12.0
1,0.066,NaN,2265,Devil's Cup,American Pale Ale (APA),177,12.0
2,0.071,NaN,2264,Rise of the Phoenix,American IPA,177,12.0
3,0.090,NaN,2263,Sinister,American Double / Imperial IPA,177,12.0
4,0.075,NaN,2262,Sex and Candy,American IPA,177,12.0
...,...,...,...,...,...,...,...
2405,0.067,45.0,928,Belgorado,Belgian IPA,424,12.0
2406,0.052,NaN,807,Rail Yard Ale,American Amber / Red Ale,424,12.0
2407,0.055,NaN,620,B3K Black Lager,Schwarzbier,424,12.0
2408,0.055,40.0,145,Silverback Pale Ale,American Pale Ale (APA),424,12.0


In [14]:
final = df1.merge(df2, on='name')

In [15]:
final.drop(columns = 'brewery_id_x', inplace=True)
final.drop(columns = 'brewery_id_y', inplace=True)
final.drop(columns = 'beer_style', inplace=True)
final.drop(columns = 'beer_abv', inplace=True)
final.drop(columns = 'beer_beerid', inplace=True)

In [16]:
#Create User ID
final['user_id'] = final.review_profilename.astype('category').cat.codes

In [17]:
final

,brewery_name,review_time,review_overall,review_aroma,review_appearance,review_profilename,review_palate,review_taste,name,abv,ibu,id,style,ounces,user_id
0,Caldera Brewing Company,1251327677,4.0,3.5,3.5,NJpadreFan,4.0,4.0,Caldera Pale Ale,0.056,55.0,1419,American Pale Ale (APA),12.0,2966
1,Caldera Brewing Company,1250928902,2.5,3.0,3.5,vacax,3.5,2.5,Caldera Pale Ale,0.056,55.0,1419,American Pale Ale (APA),12.0,10492
2,Caldera Brewing Company,1249866208,4.0,3.5,4.0,d0ggnate,4.0,3.5,Caldera Pale Ale,0.056,55.0,1419,American Pale Ale (APA),12.0,5938
3,Caldera Brewing Company,1249847121,4.5,3.5,4.0,babyhobbes,3.5,4.0,Caldera Pale Ale,0.056,55.0,1419,American Pale Ale (APA),12.0,4856
4,Caldera Brewing Company,1249556277,4.5,3.5,4.0,mdagnew,4.0,4.0,Caldera Pale Ale,0.056,55.0,1419,American Pale Ale (APA),12.0,8260
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99319,Southern Star Brewing Company,1325984869,4.0,4.0,4.0,TenHornsProud,4.0,4.0,Le Mort Vivant,0.069,23.0,1740,Bière de Garde,12.0,4039
99320,Southern Star Brewing Company,1325812388,3.5,3.5,3.5,twiggamortis420,4.5,3.5,Le Mort Vivant,0.069,23.0,1740,Bière de Garde,12.0,10432
99321,Southern Star Brewing Company,1325285639,4.0,3.5,4.0,Mora2000,4.0,3.5,Le Mort Vivant,0.069,23.0,1740,Bière de Garde,12.0,2878
99322,Southern Star Brewing Company,1324337086,4.5,4.0,4.5,HopsKeepDroppin,4.5,4.0,Le Mort Vivant,0.069,23.0,1740,Bière de Garde,12.0,1954


In [18]:
finalf = final[['user_id', 'id', 'review_taste' ]]

In [19]:
final.columns

Index(['brewery_name', 'review_time', 'review_overall', 'review_aroma',
       'review_appearance', 'review_profilename', 'review_palate',
       'review_taste', 'name', 'abv', 'ibu', 'id', 'style', 'ounces',
       'user_id'],
      dtype='object')

In [20]:
beers = final[['id', 'brewery_name', 'name', 'abv', 'ibu', 'style', 'ounces' ]]

In [21]:
beers

,id,brewery_name,name,abv,ibu,style,ounces
0,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
1,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
2,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
3,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
4,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
...,...,...,...,...,...,...,...
99319,1740,Southern Star Brewing Company,Le Mort Vivant,0.069,23.0,Bière de Garde,12.0
99320,1740,Southern Star Brewing Company,Le Mort Vivant,0.069,23.0,Bière de Garde,12.0
99321,1740,Southern Star Brewing Company,Le Mort Vivant,0.069,23.0,Bière de Garde,12.0
99322,1740,Southern Star Brewing Company,Le Mort Vivant,0.069,23.0,Bière de Garde,12.0


In [22]:
beers = beers.drop_duplicates(subset='id')

In [23]:
beers.fillna(0)

,id,brewery_name,name,abv,ibu,style,ounces
0,1419,Caldera Brewing Company,Caldera Pale Ale,0.056,55.0,American Pale Ale (APA),12.0
163,1808,Caldera Brewing Company,Pilot Rock Porter,0.060,0.0,American Porter,12.0
172,36,Caldera Brewing Company,Caldera IPA,0.061,94.0,American IPA,12.0
553,878,Caldera Brewing Company,Lawnmower Lager,0.039,16.0,American Adjunct Lager,12.0
571,47,7 Seas Brewery and Taproom,Ballz Deep Double IPA,0.084,82.0,American Double / Imperial IPA,16.0
...,...,...,...,...,...,...,...
98253,2588,Twisted X Brewing Company,Cow Creek,0.054,26.0,American Amber / Red Lager,12.0
98254,46,Southern Star Brewing Company,Buried Hatchet Stout,0.083,50.0,Foreign / Export Stout,12.0
98610,2387,Southern Star Brewing Company,Pine Belt Pale Ale,0.065,45.0,American Pale Ale (APA),12.0
98611,45,Southern Star Brewing Company,Pine Belt Pale Ale,0.065,45.0,American Pale Ale (APA),16.0


In [24]:
ratings = final[['id', 'user_id', 'review_profilename', 'review_taste']]

In [25]:
ratings

,id,user_id,review_profilename,review_taste
0,1419,2966,NJpadreFan,4.0
1,1419,10492,vacax,2.5
2,1419,5938,d0ggnate,3.5
3,1419,4856,babyhobbes,4.0
4,1419,8260,mdagnew,4.0
...,...,...,...,...
99319,1740,4039,TenHornsProud,4.0
99320,1740,10432,twiggamortis420,3.5
99321,1740,2878,Mora2000,3.5
99322,1740,1954,HopsKeepDroppin,4.0


# Base Model

In [26]:
reader = Reader(rating_scale=(0, 9))
data = Dataset.load_from_df(final[['user_id', 'id', 'review_taste']], reader)

In [158]:
benchmark = []
# Iterate over all algorithms
for algorithm in [SVD(), SVDpp(), SlopeOne(), NMF(), NormalPredictor(), KNNBaseline(), KNNBasic(), KNNWithMeans(), KNNWithZScore(), BaselineOnly(), CoClustering()]:
    # Perform cross validation
    results = cross_validate(algorithm, data, measures=['RMSE'], cv=3, verbose=False)
    
    # Get results & append algorithm name
    tmp = pd.DataFrame.from_dict(results).mean(axis=0)
    tmp = tmp.append(pd.Series([str(algorithm).split(' ')[0].split('.')[-1]], index=['Algorithm']))
    benchmark.append(tmp)

Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Estimating biases using als...


In [159]:
surprise_results = pd.DataFrame(benchmark).set_index('Algorithm').sort_values('test_rmse')

In [160]:
surprise_results

,test_rmse,fit_time,test_time
Algorithm,,,
KNNBaseline,0.509149,6.063037,20.706825
KNNBasic,0.521101,5.201628,18.571279
SVDpp,0.540642,35.910117,1.683987
SVD,0.545710,4.533566,0.265070
BaselineOnly,0.557884,0.183069,0.241880
SlopeOne,0.568368,0.223599,1.115458
KNNWithMeans,0.573541,6.068566,19.364198
KNNWithZScore,0.577555,6.760664,19.987516
CoClustering,0.635565,2.190520,0.350080


In [161]:
print('Using ALS')
bsl_options = {'method': 'als',
               'n_epochs': 5,
               'reg_u': 12,
               'reg_i': 5
               }
algo = NormalPredictor()
cross_validate(algo, data, measures=['RMSE'], cv=3, verbose=False)

Using ALS


{'test_rmse': array([0.93010345, 0.92585742, 0.92974652]),
 'fit_time': (0.09896302223205566, 0.1257801055908203, 0.1149911880493164),
 'test_time': (0.20893502235412598, 0.3811991214752197, 0.3057849407196045)}

In [162]:
trainset, testset = train_test_split(data, test_size=0.25)
algo = NormalPredictor()
predictions = algo.fit(trainset).test(testset)
accuracy.rmse(predictions)

RMSE: 0.9378


0.9378156610036569

In [163]:
trainset = algo.trainset
print(algo.__class__.__name__)

NormalPredictor


In [164]:
#Pandas DataFrame with all Predictions
def get_Iu(uid):
    """ return the number of items rated by given user
    args: 
      uid: the id of the user
    returns: 
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0
    
def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try: 
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:
        return 0
    
df = pd.DataFrame(predictions, columns=['uid', 'iid', 'rui', 'est', 'details'])
df['Iu'] = df.uid.apply(get_Iu)
df['Ui'] = df.iid.apply(get_Ui)
df['err'] = abs(df.est - df.rui)

In [165]:
df

,uid,iid,rui,est,details,Iu,Ui,err
0,5525,1813,5.0,4.642462,{'was_impossible': False},3,1091,0.357538
1,8456,901,4.0,3.637606,{'was_impossible': False},19,826,0.362394
2,6447,2050,3.5,3.761649,{'was_impossible': False},15,146,0.261649
3,3327,1411,4.5,3.320715,{'was_impossible': False},55,466,1.179285
4,7565,1905,4.5,2.194504,{'was_impossible': False},7,1971,2.305496
...,...,...,...,...,...,...,...,...
24826,9591,1379,3.5,4.135883,{'was_impossible': False},19,862,0.635883
24827,6398,70,3.5,4.188242,{'was_impossible': False},22,112,0.688242
24828,1971,45,3.5,3.301961,{'was_impossible': False},20,265,0.198039
24829,4089,1144,3.5,2.787839,{'was_impossible': False},32,1126,0.712161


In [166]:
best_predictions = df.sort_values(by='err')[:10]
worst_predictions = df.sort_values(by='err')[-10:]

In [167]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
2744,7524,1813,4.0,3.999915,{'was_impossible': False},1,1091,0.000085
9585,804,901,4.5,4.499840,{'was_impossible': False},177,826,0.000160
13158,3891,1905,3.5,3.499763,{'was_impossible': False},31,1971,0.000237
13498,7823,1813,5.0,5.000275,{'was_impossible': False},2,1091,0.000275
12513,10565,1924,3.5,3.500339,{'was_impossible': False},19,269,0.000339
24350,1713,1220,3.5,3.500344,{'was_impossible': False},50,354,0.000344
18234,9638,1411,3.0,3.000353,{'was_impossible': False},36,466,0.000353
1356,10664,1065,4.5,4.499599,{'was_impossible': False},16,1097,0.000401
14229,5740,1924,3.5,3.499593,{'was_impossible': False},16,269,0.000407
11829,375,1427,4.0,3.999530,{'was_impossible': False},4,1331,0.000470


In [168]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
24331,1621,814,1.0,4.725767,{'was_impossible': False},1,549,3.725767
6603,7847,1144,1.0,4.726408,{'was_impossible': False},5,1126,3.726408
18457,4496,539,1.0,4.737522,{'was_impossible': False},127,1128,3.737522
20591,6926,583,2.0,5.753132,{'was_impossible': False},0,784,3.753132
955,4639,1813,1.0,4.862457,{'was_impossible': False},1,1091,3.862457
848,111,2570,1.0,4.959383,{'was_impossible': False},3,65,3.959383
8947,3514,814,2.0,5.962584,{'was_impossible': False},10,549,3.962584
13464,1072,814,1.0,5.270140,{'was_impossible': False},1,549,4.270140
11554,3420,515,1.0,5.293847,{'was_impossible': False},4,496,4.293847
12869,5292,360,1.0,5.316628,{'was_impossible': False},15,1076,4.316628


# Matrix Factorization SVD

In [96]:
# Calculate the number of unique users and beers
n_users = finalf.user_id.unique().shape[0] 
n_beer = finalf.id.unique().shape[0]

In [97]:
finalf.columns

Index(['user_id', 'id', 'review_overall'], dtype='object')

In [169]:
Ratings = finalf.reset_index().pivot_table(index = 'user_id', columns ='id', values = 'review_taste').fillna(0)
Ratings.head()

id,1,6,7,11,12,13,14,15,16,17,...,2639,2642,2649,2656,2662,2663,2666,2670,2686,2688
user_id,,,,,,,,,,,,,,,,,,,,,
-1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,3.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [170]:
R = Ratings.as_matrix()
user_ratings_mean = np.mean(R, axis = 1)
Ratings_demeaned = R - user_ratings_mean.reshape(-1, 1)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [171]:
U, sigma, Vt = svds(Ratings_demeaned, k = 50)

In [172]:
sigma = np.diag(sigma)

In [173]:
all_user_predicted_ratings = np.dot(np.dot(U, sigma), Vt) + user_ratings_mean.reshape(-1, 1)

In [174]:
preds = pd.DataFrame(all_user_predicted_ratings, columns = Ratings.columns)

In [175]:
def recommend_beers(predictions, user_id, beers, original_ratings, num_recommendations):
    
    # Get and sort the user's predictions
    user_row_number = user_id - 1 # User ID starts at 1, not 0
    sorted_user_predictions = preds.iloc[user_row_number].sort_values(ascending=False) # User ID starts at 1
    
    # Get the user's data and merge in the movie information.
    user_data = original_ratings[original_ratings.user_id == (user_id)]
    user_full = (user_data.merge(beers, how = 'left', left_on = 'id', right_on = 'id').
                     sort_values(['id'], ascending=False)
                 )

    
    # Recommend the highest predicted rating movies that the user hasn't seen yet.
    recommendations = (beers[~beers['id'].isin(user_full['id'])].
         merge(pd.DataFrame(sorted_user_predictions).reset_index(), how = 'left',
               left_on = 'id',
               right_on = 'id').
         rename(columns = {user_row_number: 'Predictions'}).
         sort_values('Predictions', ascending = False).
                       iloc[:num_recommendations, :-1]
                      )

    return user_full, recommendations

In [176]:
already_rated, predictions = recommend_beers(preds, 2966, beers, ratings, 5)

In [177]:
already_rated

,id,user_id,review_profilename,review_taste,brewery_name,name,abv,ibu,style,ounces
50,2649,2966,NJpadreFan,4.0,Great Divide Brewing Company,Hibernation Ale,0.087,NaN,Old Ale,12.0
40,2587,2966,NJpadreFan,4.5,The Big Cheese Pizza Co.,Vanilla Porter,0.070,11.0,American Porter,16.0
63,2584,2966,NJpadreFan,4.0,C.H. Evans Brewing Company,Gose,0.046,8.0,Gose,16.0
64,2580,2966,NJpadreFan,4.0,C.H. Evans Brewing Company,Gose,0.035,NaN,Gose,16.0
2,2578,2966,NJpadreFan,3.0,Destiny Brewing Company,IPA,0.065,NaN,American IPA,12.0
...,...,...,...,...,...,...,...,...,...,...
70,13,2966,NJpadreFan,4.0,Surly Brewing Company,CynicAle,0.067,33.0,Saison / Farmhouse Ale,16.0
71,12,2966,NJpadreFan,5.0,Surly Brewing Company,Furious,0.062,99.0,American IPA,16.0
110,11,2966,NJpadreFan,3.0,21st Amendment Brewery,Monk's Blood,0.083,35.0,Belgian Dark Ale,12.0
108,7,2966,NJpadreFan,3.0,Oskar Blues Grill & Brew,Mama's Little Yella Pils,0.053,35.0,Czech Pilsener,12.0


In [178]:
predictions

,id,brewery_name,name,abv,ibu,style,ounces
473,432,21st Amendment Brewery,Hop Crisis,0.097,94.0,American Double / Imperial IPA,12.0
475,756,Brooklyn Brewery,Brooklyn Summer Ale,0.045,NaN,English Pale Mild Ale,12.0
338,1714,Big Sky Brewing Company,Big Sky IPA,0.062,65.0,American IPA,12.0
60,2615,Southern Tier Brewing Company,Pale,0.054,37.0,American Pale Ale (APA),16.0
73,1690,New Belgium Brewing,Sunshine Wheat Beer,0.048,NaN,American Pale Wheat Ale,12.0


# Evaluation

In [29]:
svd = SVD()

# Compute the RMSE of the SVD algorithm.
evaluate(svd, data, measures=['RMSE'])

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/surprise/evaluate.py:66: UserWarning: The evaluate() method is deprecated. Please use model_selection.cross_validate() instead.
  'model_selection.cross_validate() instead.', UserWarning)
/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/surprise/dataset.py:193: UserWarning: Using data.split() or using load_from_folds() without using a CV iterator is now deprecated. 
  UserWarning)


Evaluating RMSE of algorithm SVD.

------------
Fold 1
RMSE: 0.5321
------------
Fold 2
RMSE: 0.5364
------------
Fold 3
RMSE: 0.5398
------------
Fold 4
RMSE: 0.5384
------------
Fold 5
RMSE: 0.5379
------------
------------
Mean RMSE: 0.5369
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'rmse': [0.5320909853430009,
                             0.536368652194014,
                             0.5397643948071548,
                             0.5384331834591688,
                             0.5378879407789224]})

In [30]:
# Compute the RMSE of the SVD algorithm.
evaluate(svd, data, measures=['MAE'])



Evaluating MAE of algorithm SVD.

------------
Fold 1
MAE:  0.3942
------------
Fold 2
MAE:  0.4009
------------
Fold 3
MAE:  0.4016
------------
Fold 4
MAE:  0.3992
------------
Fold 5
MAE:  0.3992
------------
------------
Mean MAE : 0.3990
------------
------------


CaseInsensitiveDefaultDict(list,
                           {'mae': [0.3941979197923898,
                             0.40094410140577924,
                             0.4016337998381737,
                             0.39918909815831694,
                             0.3991938470423569]})

In [31]:
trainset = data.build_full_trainset()
svd.train(trainset)

/opt/anaconda3/envs/learn-env/lib/python3.6/site-packages/surprise/prediction_algorithms/algo_base.py:51: UserWarning: train() is deprecated. Use fit() instead
  warnings.warn('train() is deprecated. Use fit() instead', UserWarning)


In [33]:
svd.predict(1621, 1814)

Prediction(uid=1621, iid=1814, r_ui=None, est=3.638043372973499, details={'was_impossible': False})